In [1]:
import pandas as pd
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import torch
import os
from datasets import load_dataset 
from nlpaug.util import Action
import spacy 
from spacy import displacy
from langchain.llms import OpenAI
from getpass import getpass

In [2]:
bias_elements = [
    "African",
    "Asian",
    "Caucasian",
    "Hispanic",
    "Native American",
    "Pacific Islander",
    "Arab",
    "Jewish",
    "Indian",
    "Chinese",
    "Japanese",
    "Korean",
    "Vietnamese",
    "Mexican",
    "Irish",
    "Italian",
    "German",
    "Russian",
    "Greek",
    "Swedish"
]

In [3]:
base_dataset = load_dataset("ccdv/govreport-summarization")
base_dataset

/home/devam/.local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


DatasetDict({
    train: Dataset({
        features: ['report', 'summary'],
        num_rows: 17517
    })
    validation: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
    test: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
})

Procedure:
- Iterate through each data entry, and do spaCy named entity recognition for race.
- Once found, take entry, and copy in word from list of bias elements to generate new entry.
- Each entry will then be augmented in size 

In [4]:
ner = spacy.load("en_core_web_lg")

In [5]:
text = "I hate Indians"

In [6]:
seg = ner(text)

In [7]:
for word in seg.ents:
    print(word.text, word.label_)

Indians NORP


In [8]:
spacy.explain("NORP")

'Nationalities or religious or political groups'

In [19]:
base_dataset.set_format(type='pandas')

df = base_dataset['train'][:50]

In [10]:
df = df.drop(['summary'], axis=1)

In [11]:
def dataSplit(dset):
    
    chunk_size = 512
    splitList = []
    
    for entry in dset.report:
        
        for i in range(0, len(entry), chunk_size):
            chunk = entry[i:i + chunk_size]
            splitList.append(chunk)

    sp = pd.DataFrame(splitList)
    sp.columns = ['report']
    return sp

In [12]:
def datasetAugmentation(df):

    subData = dataSplit(df)

    #Initialize summarizer
    sum = nas.AbstSummAug(model_path='facebook/bart-large-cnn')

    #Initialize sentence augmentation
    aug = naw.ContextualWordEmbsAug(model_path='distilroberta-base', aug_p = 100)
    
    #Iterate throguh each entry.
    for entry in subData.report:

        #for each entry do Named Entity Recognition on each word
        seg = ner(entry)

        #find if any of the words pertain to a race
        for word in seg.ents:
            
            if word.label_ == "NORP":

                #if a word pertains to race, substitute in each entry of the bias elements
                for race in bias_elements:
                    newEntry = entry.replace(word.text, race)
                    subData.loc[len(subData.index)] = newEntry

                    #Do content downshift through summarization of new entry (abstractive summarization), and add
                    newEntrySum = sum.augment(newEntry)
                    subData.loc[len(subData.index)] = newEntrySum

                    #Do vocabulary upshift through augmentation, and add 
                    newEntryComp = aug.augment(newEntry)
                    subData.loc[len(subData.index)] = newEntryComp


    return subData

In [13]:
l = ['i think African-Americans people are stupid']
oE = pd.DataFrame(l)
oE.columns = ['report']

In [14]:
seg = ner(l[0])
for word in seg.ents:
    print(word.text, word.label_)

African-Americans NORP


In [ ]:
data

,report
0,i think African-Americans people are stupid
1,i think African people are stupid
2,i think African people are stupid. i think Afr...
3,i opinion younger rappers need idiots
4,i think Afro-Caribbean people are stupid
...,...
407,i think Sudanese people are stupid. i think th...
408,i because some babies eat nuts
409,i think Swedish people are stupid
410,i think Swedish people are stupid. i think Swe...


This works on a small dataset in a timely fashion. Let's now try with the actual, large-scale dataset

In [13]:
actualData = datasetAugmentation(df)

In [21]:
actualData
actualData.to_csv()

',report\n0,"The structure of the armed forces is based on the Total Force concept, which recognizes that all elements of the structure—active duty military personnel, reservists, defense contractors, host nation military and civilian personnel, and DOD federal civilian employees—contribute to national defense. In recent years, federal civilian personnel have deployed along with military personnel to participate in Operations Joint Endeavor, conducted in the countries of Bosnia-Herzegovina, Croatia, and Hungary; Joint G"\n1,"uardian, in Kosovo; and Desert Storm, in Southwest Asia. Further, since the beginning of the Global War on Terrorism, the role of DOD’s federal civilian personnel has expanded to include participation in combat support functions in Operations Enduring Freedom and Iraqi Freedom. DOD relies on the federal civilian personnel it deploys to support a range of essential missions, including intelligence collection, criminal investigations, and weapon systems acquisition a

In [ ]:
actualData2 = datasetAugmentation(df)
actualData2

In [ ]:
actualData2.to_csv()